In [747]:
import pandas

In [748]:
# visits
v1 = pandas.read_csv("C:/Users/umihu/OneDrive - Newcastle University/Documents/CSC3432/CW - part 2/data/tabular/visit-1.csv")
v2 = pandas.read_csv("C:/Users/umihu/OneDrive - Newcastle University/Documents/CSC3432/CW - part 2/data/tabular/visit-2.csv")
v3 = pandas.read_csv("C:/Users/umihu/OneDrive - Newcastle University/Documents/CSC3432/CW - part 2/data/tabular/visit-3.csv")
v4 = pandas.read_csv("C:/Users/umihu/OneDrive - Newcastle University/Documents/CSC3432/CW - part 2/data/tabular/visit-4.csv")
v5 = pandas.read_csv("C:/Users/umihu/OneDrive - Newcastle University/Documents/CSC3432/CW - part 2/data/tabular/visit-5.csv")

In [749]:
# merging all of the visits into one dataframe
complete_data = pandas.concat([v1,v2,v3,v4,v5])

In [750]:
# numeric columns
numeric_columns= ['ID', 'visit', 'delay', 'age', 'YOE', 'SES', 'MMSE', 'eTIV', 'nWBV', 'ASF']
for column in numeric_columns:
    complete_data[column] = pandas.to_numeric(complete_data[column], errors='coerce')
    
# string columns
complete_data[['MRI_ID', 'sex', 'hand', 'CDR']] = complete_data[['MRI_ID', 'sex', 'hand', 'CDR']].astype(str)

In [751]:
# ID != 'N/A'
complete_data = complete_data[complete_data['ID'].notna()]

# any empty information = '0'
complete_data = complete_data.fillna(value=0)

In [752]:
# make a temporary dataframe to keep basic age for each of the ID's
temporary_data = complete_data
temporary_data = temporary_data.groupby(['ID'])['age'].min().reset_index()
temporary_data.rename(columns={'age':'basic_age'}, inplace=True)

# sort temporary_data by ID and complete_data by ID/MRI_ID
temporary_data = temporary_data.sort_values(by=['ID'])
complete_data = complete_data.sort_values(by=['ID', 'MRI_ID'])

# merge complete_data ID with temporary_data ID
complete_data = pandas.merge(complete_data, temporary_data, on='ID')

# replace complete_data age with basic_age + visit - 1 to get the accurate age of the patient
complete_data['age'] = complete_data['basic_age'] + complete_data["visit"] - 1

# sort complete_data by ID and then the ID's MRI_ID (ascending order)
complete_data = complete_data.sort_values(by=['ID', 'MRI_ID'])

In [753]:
# get rid of basic_age as it has no use
complete_data.drop(columns="basic_age", axis=0, inplace=True)

In [754]:
# reset ID column to get rid of gaps
for i in range(1, len(complete_data)):
    curr = complete_data.loc[i, "ID"]
    prev = complete_data.loc[i-1, "ID"]
    if curr - prev >= 1 and complete_data.loc[i, "visit"] == 1:
        complete_data.loc[i, "ID"] = prev + 1
    elif curr - prev >= 1 and complete_data.loc[i, "visit"] != 1:
        complete_data.loc[i, "ID"] = prev

In [755]:
# drop first part of MRI_ID
for i in range(0, len(complete_data)):
    complete_data.loc[i, "MRI_ID"] = complete_data.loc[i, "MRI_ID"][5:] 

# rename column MRI_ID to MRIvisit
complete_data.rename(columns = {'MRI_ID':'MRI_visit'}, inplace=True)

# get rid of visit column as it now has no use
complete_data.drop(columns="visit", axis=0, inplace=True)

In [756]:
# correct any spelling errors in the CDR column
complete_data = complete_data.replace(('very midl', 'vry mild', 'very miId'), 'very mild')
complete_data = complete_data.replace('midl', 'mild')

In [757]:
# error checker for ASF column
for i in range(0, len(complete_data)):
    if complete_data.loc[i, "ASF"] > 2:
        complete_data = complete_data.replace(complete_data.loc[i, "ASF"], complete_data.loc[i, "ASF"] / 100)

# convert ID back to int from float (i believe the error checker causes a weird bug that converts ID)
complete_data["ID"] = pandas.to_numeric(complete_data["ID"]).astype("int64")

In [758]:
# get rid of any duplicate records in the dataframe
complete_data = complete_data.drop_duplicates(keep='first')

In [759]:
complete_data

,ID,MRI_visit,delay,sex,hand,age,YOE,SES,MMSE,CDR,eTIV,nWBV,ASF
0,1,MR1,0,M,R,87,14,2.0,27.0,none,1987.0,0.696,0.883
1,1,MR2,457,M,R,88,14,2.0,30.0,none,2004.0,0.681,0.876
2,2,MR1,0,M,R,75,12,0.0,23.0,very mild,1678.0,0.736,1.046
3,2,MR2,560,M,R,76,12,0.0,28.0,very mild,1738.0,0.713,1.010
4,2,MR3,1895,M,R,77,12,0.0,22.0,very mild,1698.0,0.701,1.034
5,3,MR1,0,F,R,88,18,3.0,28.0,none,1215.0,0.710,1.444
6,3,MR2,538,F,R,89,18,3.0,27.0,none,1200.0,0.718,1.462
7,4,MR1,0,M,R,80,12,4.0,28.0,none,1689.0,0.712,1.039
8,4,MR2,1010,M,R,81,12,4.0,29.0,very mild,1701.0,0.711,1.032
9,4,MR3,1603,M,R,82,12,4.0,30.0,none,1699.0,0.705,1.033


In [764]:
# saving the clean data to a .csv file in the data folder
complete_data.to_csv("CW - part 2/data/processed.pkl.gz", encoding='utf-8', index=False)